In [15]:
# insert path containing folders of extracted image
dir_path = '/Users/atl/HKU/YEAR_4/COMP4081_FYP/extractedFrames_perVideo'

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import os
import re
import pandas as pd
from tqdm import tqdm

In [6]:
# Load Model from TF hub

os.environ['TFHUB_CACHE_DIR'] = '/Users/atl/HKU/YEAR_4/COMP4081_FYP'

module = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
input_size = 192
def movenet(input_image):
  model = module.signatures['serving_default']

  # SavedModel format expects tensor type of int32.
  input_image = tf.cast(input_image, dtype=tf.int32)
  # Run model inference.
  outputs = model(input_image)
  # Output is a [1, 1, 17, 3] tensor.
  keypoints_with_scores = outputs['output_0'].numpy()
  return keypoints_with_scores

Metal device set to: Apple M1 Pro


2023-03-10 12:09:28.451775: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 12:09:28.459270: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-10 12:09:28.459511: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
# Dictionary that maps from joint names to keypoint indices.
KEYPOINT_DICT = {
    'nose': 0,
    'left_eye': 1,
    'right_eye': 2,
    'left_ear': 3,
    'right_ear': 4,
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10,
    'left_hip': 11,
    'right_hip': 12,
    'left_knee': 13,
    'right_knee': 14,
    'left_ankle': 15,
    'right_ankle': 16
}

In [16]:
# Get all folders
folders = [entry for entry in os.listdir(dir_path) if os.path.isdir(os.path.join(dir_path, entry))]
folders = [folder for folder in folders if folder.startswith("vid_")]

# find number
vid_nos = []
for i in folders:
    vid_nos.append(int(re.findall(r'\d+', i)[0]))

print(folders)

['vid_132_extracted', 'vid_181_extracted', 'vid_148_extracted', 'vid_195_extracted', 'vid_126_extracted', 'vid_40_extracted', 'vid_89_extracted', 'vid_107_extracted', 'vid_113_extracted', 'vid_54_extracted', 'vid_169_extracted', 'vid_219_extracted', 'vid_78_extracted', 'vid_145_extracted', 'vid_151_extracted', 'vid_198_extracted', 'vid_221_extracted', 'vid_170_extracted', 'vid_200_extracted', 'vid_90_extracted', 'vid_214_extracted', 'vid_164_extracted', 'vid_46_extracted', 'vid_101_extracted', 'vid_28_extracted', 'vid_115_extracted', 'vid_52_extracted', 'vid_187_extracted', 'vid_134_extracted', 'vid_73_extracted', 'vid_120_extracted', 'vid_193_extracted', 'vid_206_extracted', 'vid_176_extracted', 'vid_31_extracted', 'vid_162_extracted', 'vid_212_extracted', 'vid_118_extracted', 'vid_143_extracted', 'vid_139_extracted', 'vid_227_extracted', 'vid_157_extracted', 'vid_194_extracted', 'vid_127_extracted', 'vid_133_extracted', 'vid_74_extracted', 'vid_180_extracted', 'vid_149_extracted', 'v

In [ ]:
# Load the input image.
for i in folders:
    imagefolder_path = os.path.join(dir_path, i)
    data = []
    for frame_no in tqdm(range(len([entry for entry in os.listdir(imagefolder_path) if os.path.isfile(os.path.join(imagefolder_path, entry))]))):
        image_path = imagefolder_path+'/vid_'+re.findall(r'\d+', i)[0]+'frame'+str(frame_no)+".jpg"
        image = tf.io.read_file(image_path)
        image = tf.image.decode_jpeg(image)
        # Resize and pad the image to keep the aspect ratio and fit the expected size.
        input_image = tf.expand_dims(image, axis=0)
        input_image = tf.image.resize_with_pad(input_image, input_size, input_size)

        # Run model inference.
        keypoints_with_scores = movenet(input_image)
        px = pd.DataFrame(keypoints_with_scores[0][0]).astype("float")
        px_flatten = px.to_numpy().flatten()
        data.append(px_flatten)
    df = pd.DataFrame(data, columns=['x0', 'y0', 'c0', 'x1', 'y1', 'c1', 'x2', 'y2', 'c2', 
                                    'x3', 'y3', 'c3', 'x4', 'y4', 'c4', 'x5', 'y5', 'c5', 
                                    'x6', 'y6', 'c6', 'x7', 'y7', 'c7', 'x8', 'y8', 'c8',
                                    'x9', 'y9', 'c9', 'x10', 'y10', 'c10', 'x11', 'y11', 'c11',
                                    'x12', 'y12', 'c12', 'x13', 'y13', 'c13', 'x14', 'y14', 'c14',
                                    'x15', 'y15', 'c15', 'x16', 'y16', 'c16'])
    df.to_csv(imagefolder_path+'/out.csv')

In [ ]:
for score_no in range(6): #no. of score subfolders; dont know how to count folders with code

    dir_path = '/Users/atl/HKU/YEAR_4/COMP4081_FYP/extractedFrames_perVideo/Deadlift/Deadlift_score_'+str(score_no) #for deadlifts
    
    
    # Get all folders
    folders = [entry for entry in os.listdir(dir_path) if os.path.isdir(os.path.join(dir_path, entry))]
    folders = [folder for folder in folders if folder.startswith("vid_")]

    # find number
    vid_nos = []
    for i in folders:
        vid_nos.append(int(re.findall(r'\d+', i)[0]))
    
    print(folders)

    # Load the input image.
    for i in folders:
        imagefolder_path = os.path.join(dir_path, i)
        data = []
        for frame_no in tqdm(range(len([entry for entry in os.listdir(imagefolder_path) if os.path.isfile(os.path.join(imagefolder_path, entry))]))):
            image_path = imagefolder_path+'/vid_'+re.findall(r'\d+', i)[0]+'frame'+str(frame_no)+".jpg"
            image = tf.io.read_file(image_path)
            image = tf.image.decode_jpeg(image)
            # Resize and pad the image to keep the aspect ratio and fit the expected size.
            input_image = tf.expand_dims(image, axis=0)
            input_image = tf.image.resize_with_pad(input_image, input_size, input_size)

            # Run model inference.
            keypoints_with_scores = movenet(input_image)
            px = pd.DataFrame(keypoints_with_scores[0][0]).astype("float")
            px_flatten = px.to_numpy().flatten()
            data.append(px_flatten)
        df = pd.DataFrame(data, columns=['x0', 'y0', 'c0', 'x1', 'y1', 'c1', 'x2', 'y2', 'c2', 
                                        'x3', 'y3', 'c3', 'x4', 'y4', 'c4', 'x5', 'y5', 'c5', 
                                        'x6', 'y6', 'c6', 'x7', 'y7', 'c7', 'x8', 'y8', 'c8',
                                        'x9', 'y9', 'c9', 'x10', 'y10', 'c10', 'x11', 'y11', 'c11',
                                        'x12', 'y12', 'c12', 'x13', 'y13', 'c13', 'x14', 'y14', 'c14',
                                        'x15', 'y15', 'c15', 'x16', 'y16', 'c16'])
        df.to_csv(imagefolder_path+'/out.csv')

[]
[]
['vid_67_extracted', 'vid_25_extracted', 'vid_96_extracted', 'vid_10_extracted', 'vid_17_extracted', 'vid_47_extracted', 'vid_66_extracted', 'vid_19_extracted', 'vid_63_extracted', 'vid_5_extracted', 'vid_94_extracted', 'vid_18_extracted', 'vid_4_extracted', 'vid_20_extracted', 'vid_34_extracted', 'vid_95_extracted']


 32%|███▏      | 2460/7754 [03:00<06:15, 14.12it/s]

In [ ]:
for score_no in range(6): #no. of score subfolders; dont know how to count folders with code

    dir_path = '/Users/atl/HKU/YEAR_4/COMP4081_FYP/extractedFrames_perVideo/Squat/Squat_score_'+str(score_no) #for squats

    # Load the input image.
    for i in folders:
        imagefolder_path = os.path.join(dir_path, i)
        data = []
        for frame_no in tqdm(range(len([entry for entry in os.listdir(imagefolder_path) if os.path.isfile(os.path.join(imagefolder_path, entry))]))):
            image_path = imagefolder_path+'/vid_'+re.findall(r'\d+', i)[0]+'frame'+str(frame_no)+".jpg"
            image = tf.io.read_file(image_path)
            image = tf.image.decode_jpeg(image)
            # Resize and pad the image to keep the aspect ratio and fit the expected size.
            input_image = tf.expand_dims(image, axis=0)
            input_image = tf.image.resize_with_pad(input_image, input_size, input_size)

            # Run model inference.
            keypoints_with_scores = movenet(input_image)
            px = pd.DataFrame(keypoints_with_scores[0][0]).astype("float")
            px_flatten = px.to_numpy().flatten()
            data.append(px_flatten)
        df = pd.DataFrame(data, columns=['x0', 'y0', 'c0', 'x1', 'y1', 'c1', 'x2', 'y2', 'c2', 
                                        'x3', 'y3', 'c3', 'x4', 'y4', 'c4', 'x5', 'y5', 'c5', 
                                        'x6', 'y6', 'c6', 'x7', 'y7', 'c7', 'x8', 'y8', 'c8',
                                        'x9', 'y9', 'c9', 'x10', 'y10', 'c10', 'x11', 'y11', 'c11',
                                        'x12', 'y12', 'c12', 'x13', 'y13', 'c13', 'x14', 'y14', 'c14',
                                        'x15', 'y15', 'c15', 'x16', 'y16', 'c16'])
        df.to_csv(imagefolder_path+'/out.csv')